In [40]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pathlib import Path

In [41]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["news_api"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_Key")
alpaca_secret_key = os.getenv("Alpaca_secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [42]:
# Set the ticker
ticker = ["GSK"]

# Set timeframe to '1D'
timeframe = "1D"

# Set date and the date for 2008
start_date = pd.Timestamp("2008-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2008-12-31", tz="America/New_York").isoformat()

# Get the entire year of 2008 worth of historical data for GSK
gsk_df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

# Display data
gsk_df.head()

GSK                              
                            open   high    low  close   volume
time                                                          
2008-08-11 00:00:00-04:00  48.72  48.88  48.39  48.46   668310
2008-08-12 00:00:00-04:00  48.26  48.26  47.66  48.03  1006341
2008-08-13 00:00:00-04:00  47.63  48.14  47.53  47.79   891118
2008-08-14 00:00:00-04:00  47.17  48.00  47.17  47.78   843431
2008-08-15 00:00:00-04:00  47.16  47.72  47.16  47.58  1203574

In [43]:
# Drop Outer Table Level
gsk_df = gsk_df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
gsk_df = gsk_df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
gsk_df.index = gsk_df.index.date

# Display sample data
gsk_df.head()

,close
2008-08-11,48.46
2008-08-12,48.03
2008-08-13,47.79
2008-08-14,47.78
2008-08-15,47.58


In [44]:
#Update Column Name
gsk_df['GSK'] = gsk_df['close']
gsk_df1 = gsk_df.drop(columns = 'close')
gsk_df1.head()

,GSK
2008-08-11,48.46
2008-08-12,48.03
2008-08-13,47.79
2008-08-14,47.78
2008-08-15,47.58


In [45]:
# Set the ticker
ticker = ["PFE"]

# Set timeframe to '1D'
timeframe = "1D"

# Set date and the date for 2008
start_date = pd.Timestamp("2008-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2008-12-31", tz="America/New_York").isoformat()

# Get the entire year of 2008 worth of historical data for PFE
pfe_df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

# Display data
pfe_df.head()

PFE                               
                            open   high    low  close    volume
time                                                           
2008-08-11 00:00:00-04:00  19.81  20.11  19.69  19.85  39508859
2008-08-12 00:00:00-04:00  19.83  20.03  19.67  19.72  33634563
2008-08-13 00:00:00-04:00  19.75  19.85  19.56  19.65  32346824
2008-08-14 00:00:00-04:00  19.68  20.09  19.51  19.79  34005676
2008-08-15 00:00:00-04:00  19.87  20.13  19.86  19.98  37347287

In [46]:
# Drop Outer Table Level
pfe_df = pfe_df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
pfe_df = pfe_df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
pfe_df.index = pfe_df.index.date

# Display sample data
pfe_df.head()

,close
2008-08-11,19.85
2008-08-12,19.72
2008-08-13,19.65
2008-08-14,19.79
2008-08-15,19.98


In [47]:
#Update Column Name
pfe_df['PFE'] = pfe_df['close']
pfe_df1 = pfe_df.drop(columns = 'close')
pfe_df1.head()

,PFE
2008-08-11,19.85
2008-08-12,19.72
2008-08-13,19.65
2008-08-14,19.79
2008-08-15,19.98


In [48]:
#Combine DataFrames
pharma_df = pd.concat([gsk_df1, pfe_df1], axis="columns", join="inner")
pharma_df.head()

,GSK,PFE
2008-08-11,48.46,19.85
2008-08-12,48.03,19.72
2008-08-13,47.79,19.65
2008-08-14,47.78,19.79
2008-08-15,47.58,19.98


In [49]:
# Use the `pct_change` function to calculate daily returns
pharma_returns = pharma_df.pct_change().dropna()

# Display sample data
pharma_returns.head()

,GSK,PFE
2008-08-12,-0.008873,-0.006549
2008-08-13,-0.004997,-0.003550
2008-08-14,-0.000209,0.007125
2008-08-15,-0.004186,0.009601
2008-08-18,-0.011559,-0.016517


In [33]:
# Fetch news about GSK in 2008
gsk_news_en = newsapi.get_everything(
    q=" GSK AND GlaxoSmithKline AND 2008",
    language="en"
)

# Show the total number of news
gsk_news_en["totalResults"]

6

In [34]:
# Fetch news about PFE in 2008
pfe_news_en = newsapi.get_everything(
    q="PFE AND Pfizer AND 2008",
    language="en"
)

# Show the total number of news
pfe_news_en["totalResults"]

3

In [35]:
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

In [38]:
# Create a DataFrame with the GSK News
gsk_en_df = create_df(gsk_news_en["articles"], "en")

# Create a DataFrame with the PFE New
pfe_en_df = create_df(pfe_news_en["articles"], "fr")

pharma_news_df = pd.concat([gsk_en_df, pfe_en_df])

In [50]:
#Save News DF to CSV for further use 
file_path = Path("Data/pharma_news.csv")
pharma_news_df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [51]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [52]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(articles):
    sentiment = []
    for day in articles:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [56]:
# Get averages of each topics sentiment
sentiment_avg = pd.DataFrame(headline_sentiment_summarizer_avg(pharma_df))

In [58]:
sentiment_avg.head()

,0
0,0.0
1,0.0


In [63]:
#Save pharma df to CSV for further use 
pharma_path = Path("Data/pharma_returns.csv")
pharma_df.to_csv(pharma_path)